In [1]:
import pandas as pd
import os
import json
import time
from tqdm import tqdm
from openai import OpenAI
import google.generativeai as genai
import anthropic


C:\Users\Klaywert\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# --- 1. CONFIGURAÇÃO ---
# Cole suas API Keys aqui
OPENAI_API_KEY = ''
GOOGLE_API_KEY = ''
ANTHROPIC_API_KEY = '' 

# Clientes das APIs
client_openai = OpenAI(api_key=OPENAI_API_KEY)
genai.configure(api_key=GOOGLE_API_KEY)
client_anthropic = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

# Nomes dos modelos
MODEL_GPT = 'gpt-4o'
MODEL_GEMINI = 'gemini-2.5-flash'
MODEL_CLAUDE = 'claude-opus-4-1-20250805'

# Definição das pastas
PROMPTS_DIR = 'prompts'
DEBATES_DIR = 'debates'
OUTPUTS_DIR = 'outputs'

# Parâmetros do experimento
NUM_PROMPTS = 4
debate_numbers = [i for i in range(1, 19) if i not in [4, 15]]
NUM_RUNS = 3

# Configurações para as chamadas das APIs (temperatura 0.0)
generation_config_google = {"temperature": 0.0}
temperature_gpt_claude = 0.0

# --- 2. FUNÇÕES AUXILIARES ---

def format_debate_csv(file_path):
    """Lê um arquivo CSV de debate e formata em uma string única."""
    try:
        df = pd.read_csv(file_path)
        df['debater'] = df['debater'].str.upper()
        combined_transcriptions = df.apply(lambda row: f"- {row['debater']}: {row['transcription']}", axis=1)
        return "\n".join(combined_transcriptions)
    except Exception as e:
        print(f"Erro ao ler ou formatar o debate {file_path}: {e}")
        return None

def extract_json_from_response(text):
    """Extrai um bloco de código JSON da resposta do LLM."""
    try:
        if '```json' in text:
            start = text.find('```json') + len('```json')
            end = text.rfind('```')
            if start < end:
                json_str = text[start:end].strip()
                return json.loads(json_str)
        return json.loads(text)
    except json.JSONDecodeError:
        print(f"AVISO: A resposta não é um JSON válido. Salvando como texto bruto.\nResposta recebida:\n{text[:200]}...")
        return {"error": "Invalid JSON response", "raw_response": text}
    except Exception as e:
        print(f"Erro inesperado ao extrair JSON: {e}")
        return {"error": f"Unexpected error extracting JSON: {e}", "raw_response": text}

def call_api(model_name, prompt):
    """Função genérica para chamar as diferentes APIs."""
    try:
        if model_name == MODEL_GPT:
            response = client_openai.chat.completions.create(
                model=model_name,
                temperature=temperature_gpt_claude,
                messages=[{"role": "user", "content": prompt}]
            )
            return response.choices[0].message.content.strip()
        elif model_name == MODEL_GEMINI:
            model = genai.GenerativeModel(model_name)
            response = model.generate_content(prompt, generation_config=generation_config_google)
            return response.text
        elif model_name == MODEL_CLAUDE: # <-- Lógica para o Claude
            message = client_anthropic.messages.create(
                model=model_name,
                max_tokens=4096, # Claude requer um max_tokens
                temperature=temperature_gpt_claude,
                messages=[{"role": "user", "content": prompt}]
            )
            return message.content[0].text
    except Exception as e:
        print(f"Erro na chamada da API para o modelo {model_name}: {e}")
        return None

# --- 3. LÓGICA PRINCIPAL DA BATERIA DE TESTES ---

if __name__ == "__main__":
    prompts_content = {}
    for i in range(1, NUM_PROMPTS + 1):
        prompt_path = os.path.join(PROMPTS_DIR, f'prompt{i}.txt')
        try:
            with open(prompt_path, 'r', encoding='utf-8') as f:
                prompts_content[i] = f.read()
            print(f"Prompt {i} carregado.")
        except FileNotFoundError:
            print(f"ERRO CRÍTICO: Arquivo de prompt não encontrado: {prompt_path}")
            exit()

    os.makedirs(OUTPUTS_DIR, exist_ok=True)
    
    models_to_run = {
        #"gpt4o": MODEL_GPT,
        #"gemini": MODEL_GEMINI,
        "claude": MODEL_CLAUDE
    }
    
    total_api_calls = len(prompts_content) * len(debate_numbers) * NUM_RUNS * len(models_to_run)
    pbar = tqdm(total=total_api_calls, desc="Executando Bateria de Testes")

    for prompt_num in range(1, NUM_PROMPTS + 1):
        for debate_num in debate_numbers:
            for run_num in range(1, NUM_RUNS + 1):
                
                output_folder = os.path.join(OUTPUTS_DIR, f'prompt_{prompt_num}', f'debate_{debate_num}')
                os.makedirs(output_folder, exist_ok=True)
                
                debate_path = os.path.join(DEBATES_DIR, f'debate{debate_num}.csv')
                debate_text = format_debate_csv(debate_path)
                if debate_text is None:
                    pbar.update(len(models_to_run))
                    continue

                prompt_template = prompts_content[prompt_num]
                full_prompt = f"{prompt_template}\n\nDEBATE:\n{debate_text}"

                for model_key, model_name in models_to_run.items():
                    output_path = os.path.join(output_folder, f'{model_key}_run_{run_num}.json')
                    
                    if not os.path.exists(output_path):
                        response = call_api(model_name, full_prompt)
                        if response:
                            json_data = extract_json_from_response(response)
                            with open(output_path, 'w', encoding='utf-8') as f:
                                json.dump(json_data, f, ensure_ascii=False, indent=2)
                        time.sleep(1) 
                    
                    pbar.update(1)
    
    pbar.close()
    print("\nBateria de testes completa!")

Prompt 1 carregado.
Prompt 2 carregado.
Prompt 3 carregado.
Prompt 4 carregado.


Executando Bateria de Testes:  97%|█████████▋| 186/192 [08:27<02:39, 26.56s/it] 

AVISO: A resposta não é um JSON válido. Salvando como texto bruto.
Resposta recebida:
```json
{
  "debaters": [
    {
      "name": "Debater 1",
      "evaluation_aspects": {
        "organization_and_clarity": {
          "score": 4,
          "occurrences": [
            "Initial sta...


Executando Bateria de Testes:  98%|█████████▊| 188/192 [11:44<03:05, 46.37s/it]

AVISO: A resposta não é um JSON válido. Salvando como texto bruto.
Resposta recebida:
```json
{
  "debaters": [
    {
      "name": "Debater 1",
      "evaluation_aspects": {
        "organization_and_clarity": {
          "score": 4,
          "occurrences": [
            "Initial sta...


...

AVISO: A resposta não é um JSON válido. Salvando como texto bruto.
Resposta recebida:
```json
{
  "debaters": [
    {
      "name": "Debater 1",
      "evaluation_aspects": {
        "organization_and_clarity": {
          "score": 3,
          "occurrences": [
            "Opening sta...


Executando Bateria de Testes:  99%|█████████▉| 190/192 [15:02<02:10, 65.33s/it]

AVISO: A resposta não é um JSON válido. Salvando como texto bruto.
Resposta recebida:
```json
{
  "debaters": [
    {
      "name": "Debater 1",
      "evaluation_aspects": {
        "organization_and_clarity": {
          "score": 3,
          "occurrences": [
            "Opening sta...


Executando Bateria de Testes: 100%|██████████| 192/192 [18:24<00:00,  5.75s/it]


Bateria de testes completa!
